# **SEMI-SUPERVISED LEARNING ON OVERHEAD MNIST DATASET**

In [ ]:
!pip install ipython-autotime --quiet
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 6.95 s (started: 2024-04-21 16:12:31 +00:00)


#  Import Necessary Models

In [ ]:
# import necessary models
from tqdm import tqdm
import pandas as pd
import numpy as np
from tensorflow import keras
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.linear_model import LogisticRegression

time: 993 µs (started: 2024-04-21 16:12:38 +00:00)


# Load Overhead MNIST data

In [ ]:
# Save files as dataframes

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

time: 909 ms (started: 2024-04-21 16:12:38 +00:00)


# Data Preprocessing

In [ ]:
X_train = train.iloc[:, 1:].to_numpy()
y_train = train.iloc[:, 0].to_numpy()

time: 2.18 ms (started: 2024-04-21 16:12:39 +00:00)


In [ ]:
X_test = test.iloc[:, 1:].to_numpy()
y_test = test.iloc[:, 0].to_numpy()

time: 1.88 ms (started: 2024-04-21 16:12:39 +00:00)


In [ ]:
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

time: 53.8 ms (started: 2024-04-21 16:12:39 +00:00)


# A function to print a particular image

In [ ]:
get_original_array = lambda arr: np.array([int(x * 255) for x in arr], dtype=np.uint8)

def print_image(i, data='train'):
  if data == 'train':
    return get_original_array(X_train[i]).reshape(28,28)
  else:
    return get_original_array(X_test[i]).reshape(28,28)

#example
print_image(0)

array([[ 86,  91,  98,  97,  96,  89,  76,  84,  87,  96,  85,  78,  60,
         71,  78,  85,  91,  96, 106,  97,  74, 108, 126, 100,  54,  72,
         68,  93],
       [ 81,  87,  89, 100, 104,  91, 107,  97,  91,  92,  90,  88,  94,
         87,  76,  96,  99,  99,  95,  94, 112, 127, 109, 114, 132,  95,
         92, 113],
       [ 81,  67,  81,  86,  89,  91,  96,  87,  89,  83, 107, 105,  83,
        103, 145, 107,  93, 102, 105,  99,  82,  56,  31,  42,  61,  88,
        101, 120],
       [ 38,  48,  80,  98,  92,  86,  86,  80,  72,  78, 143, 124,  75,
        103, 135,  88,  78,  85,  70,  88,  80,  25,  21,  60,  97,  87,
         93, 130],
       [ 81,  72,  96,  86,  88,  78,  54,  53,  60,  75,  61,  62,  79,
         74,  73,  73,  79, 110, 137, 155, 118,  53,  42,  48,  88,  80,
         85, 121],
       [ 90, 103,  92,  79,  91,  73, 120, 136,  79,  64,  73,  80,  72,
         84,  90,  98,  93, 122, 174, 161, 112, 104, 107,  80,  70, 200,
        181, 144],
       [ 99, 100,  82,  75,  90,  88, 119, 133,  83,  90,  80,  76,  72,
         79,  87,  84,  92,  81,  84,  71,  77,  90,  88,  83,  63, 246,
        241, 169],
       [107, 104,  75,  87,  84,  84,  77,  70,  78,  76,  77,  76,  76,
         77,  76,  81,  79,  74,  85,  87,  84,  81,  98, 102,  49, 206,
        255, 178],
       [ 99, 102,  76,  84,  81,  76,  83,  79,  81,  76,  72,  75,  79,
         77,  77,  82,  78,  80,  83,  78,  83,  81,  90,  96,  54, 168,
        250, 188],
       [100, 105,  81,  84,  84,  81,  88,  85,  88,  83,  77,  77,  80,
         79,  82,  90,  88,  88,  89,  84,  86,  81,  87,  93,  87,  87,
        135, 130],
       [ 98, 107,  84,  79,  82,  82,  86,  86,  87,  86,  83,  83,  84,
         82,  83,  89,  90,  90,  91,  87,  88,  82,  86,  94,  87,  42,
         71,  81],
       [ 95, 109,  90,  76,  80,  82,  83,  85,  86,  86,  85,  85,  86,
         83,  82,  85,  86,  86,  86,  87,  87,  81,  84,  93,  81,  46,
         81, 100],
       [ 93, 109,  98,  78,  81,  84,  83,  86,  90,  89,  84,  82,  83,
         83,  83,  86,  85,  85,  83,  86,  84,  80,  80,  89,  89,  53,
         74,  87],
       [ 85, 103, 103,  80,  80,  83,  82,  85,  88,  88,  85,  82,  84,
         84,  82,  82,  85,  85,  81,  85,  83,  82,  81,  87,  89,  49,
         50,  57],
       [ 80,  97, 108,  84,  79,  80,  79,  82,  83,  87,  87,  85,  85,
         83,  80,  79,  84,  85,  79,  83,  82,  85,  82,  87,  99,  57,
         40,  60],
       [ 82,  98, 115,  90,  82,  82,  79,  82,  86,  90,  88,  83,  82,
         82,  82,  84,  85,  86,  79,  81,  80,  85,  81,  83,  89,  67,
         35,  55],
       [ 84,  93, 112,  95,  72,  85,  82,  80,  86,  86,  86,  83,  81,
         83,  84,  81,  85,  84,  80,  77,  78,  78,  78,  82,  84,  76,
         62,  69],
       [ 77,  80, 106, 103,  77,  86,  81,  84,  84,  88,  87,  85,  85,
         82,  78,  79,  84,  85,  82,  80,  82,  82,  82,  84,  99,  95,
         79, 110],
       [ 80,  88, 108, 105,  78,  89,  78,  83,  80,  88,  84,  77,  81,
         81,  79,  84,  82,  83,  80,  76,  76,  76,  75,  76,  87,  86,
         75,  70],
       [ 65,  86, 103, 104,  81,  97,  79,  75,  80,  90,  90,  83,  81,
         82,  79,  77,  80,  83,  79,  74,  74,  76,  76,  76,  87,  82,
         89,  33],
       [ 68,  73,  89, 106,  80,  91,  86,  80,  79,  79,  81,  80,  74,
         72,  74,  72,  73,  76,  75,  74,  76,  79,  79,  79,  82,  72,
         84,  33],
       [ 79,  67,  76, 111,  84,  76,  85,  82,  77,  73,  74,  76,  71,
         67,  69,  72,  72,  74,  74,  74,  75,  72,  69,  69,  80,  76,
         69,  46],
       [ 73,  90,  81, 106,  99,  75,  76,  79,  77,  88,  82,  73,  78,
         81,  77,  79,  78,  78,  76,  76,  75,  70,  67,  69,  77,  82,
         73,  60],
       [ 96, 152, 105,  88, 105,  82,  75,  90, 190, 204, 160, 101,  92,
         95,  82,  79,  74,  71,  69,  71,  74,  71,  73,  79,  73,  75,
         87,  72],
       [ 89, 107,  93,  80,  92,  95,  8

time: 15.5 ms (started: 2024-04-21 16:12:44 +00:00)


# Standard Logistic Regression

In [ ]:
# Standard logistic model has a very high accuracy for the dataset.

logistic_regressor_clf = LogisticRegression(random_state=69, max_iter=10000).fit(X_train, y_train)
logistic_regressor_clf.score(X_test, y_test)

0.4676056338028169

time: 1min 22s (started: 2024-04-21 16:12:54 +00:00)


The standard logistic regression model gives an accuracy score of 0.4676 which is pretty less.

# **Now, we will perform semi-supervised learning on the Fashion MNIST dataset and compare the accuracy scores.**

1.   First, we will take 40 random samples and do the logistic regression.
2.   Next, we will use k-means clustering to get 40 clusters and take the nearest points as centroids. Then we will use those 40 data-points to train the logistic regression classifier.
3. Then we will propagate the labelling to the whole cluster and do the same.
4. Repeat the same, but only propagate to the 20% of the dataset now.



# Perform Logistic Regression On Randomly Selected 40 Samples

In [ ]:
# Randomly select 40 samples from the training data
idx = np.random.choice(X_train.shape[0], 40, replace=False)

# Get the selected samples and their corresponding labels
X_sample = X_train[idx]
y_sample = y_train[idx]

logistic_regressor_clf = LogisticRegression(random_state=69, max_iter=10000).fit(X_sample, y_sample)
logistic_regressor_clf.score(X_test, y_test)

0.20093896713615023

time: 975 ms (started: 2024-04-21 16:29:25 +00:00)


Note that a pretty low accuracy score of 0.2009 is obtained.

# Perform Logistic Regression On 40 Cluster Representative Points

In [ ]:
# Use KMeans to select 40 representative samples and then train the classifier

kmeans = MiniBatchKMeans(n_clusters=40, random_state=42)
kmeans.fit(X_train)

cluster_centers = kmeans.cluster_centers_

nearest_points_to_cluster = [-1]*40
distance_from_nearest_points_to_cluster = [10e9]*40

# Select those representative points
for i, cluster_center in enumerate(cluster_centers):
  for j, data_point in enumerate(X_train):
    distance = np.linalg.norm(cluster_center-data_point)
    if distance < distance_from_nearest_points_to_cluster[i]:
      distance_from_nearest_points_to_cluster[i] = distance
      nearest_points_to_cluster[i] = j

X_sample_ = X_train[nearest_points_to_cluster]
y_sample_ = y_train[nearest_points_to_cluster]

# Train and test the logistic regression classifier
logistic_regressor_clf = LogisticRegression(random_state=69, max_iter=10000).fit(X_sample_, y_sample_)
logistic_regressor_clf.score(X_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


0.2516431924882629

time: 18.4 s (started: 2024-04-21 16:34:36 +00:00)


# Perform Logistic Regression On 40 Cluster Representative Points After Propagating the Labels

In [ ]:
# Use KMeans to select 40 representative samples, propagate the labels and then train the classifier

kmeans = MiniBatchKMeans(n_clusters=40, random_state=42)
kmeans.fit(X_train)

cluster_centers = kmeans.cluster_centers_
datapoints_labels = kmeans.labels_

# Nearest points of each cluster
nearest_points_to_cluster = [-1]*40
distance_from_nearest_points_to_cluster = [10e9]*40

# Mapping between labels and nearest points
nearest_point_of_a_label = [-1]*40

# Select those representative points
for i, cluster_center in enumerate(cluster_centers):
  for j, data_point in enumerate(X_train):
    distance = np.linalg.norm(cluster_center-data_point)
    if distance < distance_from_nearest_points_to_cluster[i]:
      distance_from_nearest_points_to_cluster[i] = distance
      nearest_points_to_cluster[i] = j
      nearest_point_of_a_label[datapoints_labels[j]] = j

y_propagated = [-1]*(X_train.shape[0])
for i, datapoint in enumerate(X_train):
  cluster_label = datapoints_labels[i]
  y_propagated[i] = y_train[nearest_point_of_a_label[cluster_label]]

# Train and test the logistic regression classifier
logistic_regressor_clf = LogisticRegression(random_state=69, max_iter=10000).fit(X_train, y_propagated)
logistic_regressor_clf.score(X_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


0.27417840375586855

time: 2min 54s (started: 2024-04-21 16:30:19 +00:00)


# Perform Logistic Regression On 40 Cluster Representative Points After Propagating the Labels Partially

In [ ]:
# Use KMeans to select 40 representative samples, propagate the labels partially and then train the classifier

kmeans = MiniBatchKMeans(n_clusters=40, random_state=42)
kmeans.fit(X_train)

cluster_centers = kmeans.cluster_centers_
datapoints_labels = kmeans.labels_

# Nearest points of each cluster
nearest_points_to_cluster = [-1]*40
distance_from_nearest_points_to_cluster = [10e9]*40

# Mapping between labels and nearest points
nearest_point_of_a_label = [-1]*40

# Select those representative points
for i in tqdm(range(len(cluster_centers))):
  for j, data_point in enumerate(X_train):
    distance = np.linalg.norm(cluster_centers[i]-data_point)
    if distance < distance_from_nearest_points_to_cluster[i]:
      distance_from_nearest_points_to_cluster[i] = distance
      nearest_points_to_cluster[i] = j
      nearest_point_of_a_label[datapoints_labels[j]] = j

closest_points_from_centroid = np.array([[1]*40]*40)

for i in tqdm(range(len(nearest_points_to_cluster))):
  distance_point_pair = np.empty((0, 2))

  for j, data_point in enumerate(X_train):

    dist = np.linalg.norm(data_point-X_train[nearest_points_to_cluster[i]])
    to_append = np.array([dist, j])
    distance_point_pair = np.concatenate((distance_point_pair, [to_append]))

  sorted_indices = np.argsort(distance_point_pair[:, 0])
  distance_point_pair = (distance_point_pair[sorted_indices])[:40]

  closest_points_idx = distance_point_pair[:, 1]
  closest_points_from_centroid[i] = closest_points_idx

X_sample = []
y_propagated = []

for i, id in enumerate(closest_points_from_centroid):
  for j in id:
    X_sample.append(X_train[j])
    y_propagated.append(y_train[nearest_points_to_cluster[i]])

# Train and test the logistic regression classifier
logistic_regressor_clf = LogisticRegression(random_state=37, max_iter=10000).fit(X_sample, y_propagated)
logistic_regressor_clf.score(X_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
100%|██████████| 40/40 [00:10<00:00,  3.97it/s]


0.26291079812206575

time: 46.7 s (started: 2024-04-21 16:51:02 +00:00)


## Report:

- NOTE THAT THE NUMBER OF CLUSTERS TO BE MADE WAS CHOSEN BASED ON THE SIZE OF THE DATASET.
- Standard logistic model has a moderate accuracy for the dataset. (`0.4676056338028169`)
- Now, we choose randomly 40 datapoints (out of 60000) and use standard logistic regression to get (`0.20093896713615023`). The performance was poor.
- Then, instead of choosing randomly, we cluster the dataset into 40 clusters and used the nearest points to the centroids as the representative points to train the standard logistic model. The accuracy was `0.2516431924882629`. The accuracy is greatly improved as we are now using the best of its kind to train.
- Next, we propagate the labels to every cluster point to get the accuracy as `0.27417840375586855`. Performance improves as there are more training points.
- Now, instead of propagating to all the points, we propagate to the 20% data (`0.26291079812206575`). Surprisingly, the performance is still moderate implying labelling small dataset accordingly (semi-supervised learning) performs greatly.